In [1]:
#  article dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as Datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
# import cv2
from tqdm.notebook import tqdm
from tqdm import tqdm as tqdm_regular
import seaborn as sns
from torchvision.utils import make_grid
import random
from torchinfo import summary
 # Input shape (3 channels, 32x32 pixels)


In [2]:
#  configuring device
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Running on the GPU')
else:
  device = torch.device('cpu')
  print('Running on the CPU')

Running on the GPU


In [3]:
import os
import gc
import cv2
import math
import copy
import time
import random
import glob
from matplotlib import pyplot as plt

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.model_selection import train_test_split

# For descriptive error messages
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
df = pd.read_csv("/home/21je0963/Competiton/data/new_train.csv")
# df

In [5]:
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

In [6]:
# for i in range 
train_data['label'].unique()

array(['CC', 'HGSC', 'EC', 'MC', 'LGSC'], dtype=object)

In [7]:
dictionary = val_data['label'].unique()

In [8]:
dictionary

array(['EC', 'CC', 'HGSC', 'MC', 'LGSC'], dtype=object)

In [9]:
index_to_labels = {index:labels for index, labels in enumerate(dictionary)}

In [10]:
index_to_labels

{0: 'EC', 1: 'CC', 2: 'HGSC', 3: 'MC', 4: 'LGSC'}

In [11]:
labels_to_index = {labels:index for index, labels in enumerate(dictionary)}
labels_to_index

{'EC': 0, 'CC': 1, 'HGSC': 2, 'MC': 3, 'LGSC': 4}

In [12]:
train_data.head()

,image_id,label,image_width,image_height,is_tma,img_path,fold
0,56947,CC,59288,24926,False,56947_thumbnail.png,3
1,27747,HGSC,47487,28122,False,27747_thumbnail.png,2
2,5264,HGSC,45054,22207,False,5264_thumbnail.png,0
3,27739,HGSC,80170,41514,False,27739_thumbnail.png,1
4,32432,EC,35266,20507,False,32432_thumbnail.png,4


In [13]:
def process(df):
    return labels_to_index[df]

train_labels = train_data['label'].map(process)

train_labels = list(train_labels)

# train_labels
color = torch.tensor(train_labels) # A tensor of categorical values

one_hot_color = torch.nn.functional.one_hot(color, num_classes=5) # Converts the categorical tensor into a one-hot encoded tensor


one_hot_color

tensor([[0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        ...,
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0]])

In [14]:
# train_data = train_data.drop(columns = ['label'])
train_data.head()

,image_id,label,image_width,image_height,is_tma,img_path,fold
0,56947,CC,59288,24926,False,56947_thumbnail.png,3
1,27747,HGSC,47487,28122,False,27747_thumbnail.png,2
2,5264,HGSC,45054,22207,False,5264_thumbnail.png,0
3,27739,HGSC,80170,41514,False,27739_thumbnail.png,1
4,32432,EC,35266,20507,False,32432_thumbnail.png,4


In [15]:
img_id = train_data['image_id']

train_data['label'][0] = one_hot_color[0]

for i, vect in enumerate(one_hot_color):
    train_data['label'][i] = vect

train_data = train_data.drop(columns = ['image_width', 'image_height','is_tma'])
train_data.head()

train_data['label'][0]


tensor([0, 1, 0, 0, 0])

In [16]:
def process(df):
    return labels_to_index[df]

val_labels = val_data['label'].map(process)

val_labels = list(val_labels)

# train_labels
color = torch.tensor(val_labels) # A tensor of categorical values

val_one_hot_color = torch.nn.functional.one_hot(color, num_classes=5) # Converts the categorical tensor into a one-hot encoded tensor


# val_one_hot_color

img_id = val_data['image_id']

val_data['label'][0] = val_one_hot_color[0]

for i, vect in enumerate(val_one_hot_color):
    val_data['label'][i] = vect

val_data = val_data.drop(columns = ['image_width', 'image_height','is_tma'])
val_data.head()

# # train_data['labels'][0]


,image_id,label,img_path,fold
0,6363,"[tensor(1), tensor(0), tensor(0), tensor(0), t...",6363_thumbnail.png,2
1,53098,"[tensor(1), tensor(0), tensor(0), tensor(0), t...",53098_thumbnail.png,2
2,28603,"[tensor(0), tensor(1), tensor(0), tensor(0), t...",28603_thumbnail.png,1
3,30794,"[tensor(0), tensor(0), tensor(1), tensor(0), t...",30794_thumbnail.png,2
4,11559,"[tensor(0), tensor(0), tensor(1), tensor(0), t...",11559_thumbnail.png,1


In [17]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, data, data_dir, transform=None):
        # Load the CSV file with image filenames and labels
        self.data = data
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Get the image filename and label from the CSV data
        img_filename = os.path.join(self.data_dir, self.data['img_path'][index])
        label = self.data['label'][index]
        
        with Image.open(img_filename) as img:
            img = img.convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, label

data_dir = '/home/21je0963/Competiton/data/train_img'
# csv_file = 'your_csv_file.csv'  # Replace with the actual CSV file path

transform = transforms.Compose([
    transforms.Resize((768, 768)),
    transforms.ToTensor(),
])

# Create instances of your custom dataset for training and validation
train_custom_dataset = CustomDataset(train_data, data_dir, transform=transform)

# Assuming you have a separate validation CSV file, you can create a validation dataset like this:
# val_csv_file = 'your_validation_csv_file.csv'  # Replace with the actual CSV file path
val_custom_dataset = CustomDataset(val_data, data_dir, transform=transform)


In [18]:
import matplotlib.pyplot as plt

batch_size = 4  # Specify your desired batch size
data_loader = DataLoader(train_custom_dataset, batch_size=batch_size, shuffle=True)
for batch in data_loader:
    images, labels = batch
    break
labels

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0]])

In [19]:

# encoder = Encoder()
# encoder.to(device)
# summary(encoder, (3, 32, 32), device = device) 
class Encoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),  # (768, 768)
            act_fn,
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            act_fn,
            nn.Conv2d(out_channels, 2*out_channels, 3, padding=1, stride=2),  # (384, 384)
            act_fn,
            nn.Conv2d(2*out_channels, 2*out_channels, 3, padding=1),
            act_fn,
            nn.Conv2d(2*out_channels, 4*out_channels, 3, padding=1, stride=2),  # (192, 192)
            act_fn,
            nn.Conv2d(4*out_channels, 4*out_channels, 3, padding=1),
            act_fn,
            nn.Flatten(),
            nn.Linear(4*out_channels*192*192, latent_dim),
            act_fn
        )

    def forward(self, x):
        x = x.view(-1, 3, 768, 768)
        output = self.net(x)
        return output

class Decoder(nn.Module):
    def __init__(self, in_channels=3, out_channels=16, latent_dim=200, act_fn=nn.ReLU()):
        super().__init__()

        self.out_channels = out_channels

        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 4 * out_channels * 192 * 192),
            act_fn
        )

        self.conv = nn.Sequential(
            nn.ConvTranspose2d(4 * out_channels, 4 * out_channels, 3, padding=1),  # (192, 192)
            act_fn,
            nn.ConvTranspose2d(4 * out_channels, 2 * out_channels, 3, padding=1, stride=2, output_padding=1),  # (384, 384)
            act_fn,
            nn.ConvTranspose2d(2 * out_channels, 2 * out_channels, 3, padding=1),
            act_fn,
            nn.ConvTranspose2d(2 * out_channels, out_channels, 3, padding=1, stride=2, output_padding=1),  # (768, 768)
            act_fn,
            nn.ConvTranspose2d(out_channels, out_channels, 3, padding=1),
            act_fn,
            nn.ConvTranspose2d(out_channels, in_channels, 3, padding=1)
        )

    def forward(self, x):
        output = self.linear(x)
        output = output.view(-1, 4 * self.out_channels, 192, 192)
        output = self.conv(output)
        return output



class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [20]:
model = torch.load('mse_model_weights.pth')

In [21]:
encoder = model.encoder

In [22]:
summary(encoder, (3,768,768))

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [1, 200]                  --
├─Sequential: 1-1                        [1, 200]                  --
│    └─Conv2d: 2-1                       [1, 16, 768, 768]         448
│    └─ReLU: 2-2                         [1, 16, 768, 768]         --
│    └─Conv2d: 2-3                       [1, 16, 768, 768]         2,320
│    └─ReLU: 2-4                         [1, 16, 768, 768]         --
│    └─Conv2d: 2-5                       [1, 32, 384, 384]         4,640
│    └─ReLU: 2-6                         [1, 32, 384, 384]         --
│    └─Conv2d: 2-7                       [1, 32, 384, 384]         9,248
│    └─ReLU: 2-8                         [1, 32, 384, 384]         --
│    └─Conv2d: 2-9                       [1, 64, 192, 192]         18,496
│    └─ReLU: 2-10                        [1, 64, 192, 192]         --
│    └─Conv2d: 2-11                      [1, 64, 192, 192]         36,9

In [23]:
for param in encoder.parameters():
    param.requires_grad = False


classification_head = nn.Sequential(
    nn.Linear(200, 128),  # Adjust num_hidden_units as needed
    nn.ReLU(),
    nn.Linear(128, 5)  # Adjust num_classes as needed
)

classification_model = nn.Sequential(
    encoder,
    classification_head
)

summary(classification_model, (3, 768, 768))

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [1, 5]                    --
├─Encoder: 1-1                           [1, 200]                  --
│    └─Sequential: 2-1                   [1, 200]                  --
│    │    └─Conv2d: 3-1                  [1, 16, 768, 768]         (448)
│    │    └─ReLU: 3-2                    [1, 16, 768, 768]         --
│    │    └─Conv2d: 3-3                  [1, 16, 768, 768]         (2,320)
│    │    └─ReLU: 3-4                    [1, 16, 768, 768]         --
│    │    └─Conv2d: 3-5                  [1, 32, 384, 384]         (4,640)
│    │    └─ReLU: 3-6                    [1, 32, 384, 384]         --
│    │    └─Conv2d: 3-7                  [1, 32, 384, 384]         (9,248)
│    │    └─ReLU: 3-8                    [1, 32, 384, 384]         --
│    │    └─Conv2d: 3-9                  [1, 64, 192, 192]         (18,496)
│    │    └─ReLU: 3-10                   [1, 64, 192, 192]   

In [24]:
class ConvolutionalAutoencoder():
  def __init__(self, autoencoder):
    self.network = autoencoder
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr=1e-3)
    self.file1 = open("Train_logs_classification.txt",'w')
    self.file1.write("Training_started..............\n")
    self.file1.close()
      
    
  def train(self, loss_function, epochs, batch_size, training_set, validation_set):
    #  creating log
    log_dict = {
        'training_loss_per_batch': [],
        'validation_loss_per_batch': [],
        'visualizations': []
    } 

    #  defining weight initialization function
    def init_weights(module):
      if isinstance(module, nn.Conv2d):
        torch.nn.init.xavier_uniform_(module.weight)
        module.bias.data.fill_(0.01)
      elif isinstance(module, nn.Linear):
        torch.nn.init.xavier_uniform_(module.weight)
        module.bias.data.fill_(0.01)

    #  initializing network weights
    self.network.apply(init_weights)

    #  creating dataloaders
    
    train_loader = DataLoader(training_set, batch_size=8, shuffle=True)
    val_loader = DataLoader(validation_set, batch_size=8, shuffle=True)
    # test_loader = DataLoader(test_dataset, 10)
    #  setting convnet to training mode
    self.network.train()
    self.network.to(device)

    for epoch in range(epochs):
      print(f'Epoch {epoch+1}/{epochs}')
      train_losses = []

      #------------
      #  TRAINING
      #------------
      print('training...')
      for images, labels in tqdm(train_loader):
        #  zeroing gradients
        self.optimizer.zero_grad()
        #  sending images to device
        # print(len(images))
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.float()
        #  reconstructing images
        output = self.network(images)
        # print(output.shape)
        #  computing loss
        loss = loss_function(output, labels)
        #  calculating gradients
        loss.backward()
        #  optimizing weights
        self.optimizer.step()

        #--------------
        # LOGGING
        #--------------
        log_dict['training_loss_per_batch'].append(loss.item())

      #--------------
      # VALIDATION
      #--------------
      print('validating...')
      for val_images, val_labels in tqdm(val_loader):
      
        with torch.no_grad():
          #  sending validation images to device
          val_images = val_images.to(device)
          val_labels = val_labels.to(device)
          val_labels = val_labels.float()
          #  reconstructing images
          output = self.network(val_images)
          #  computing validation loss
          val_loss = loss_function(output, val_labels)

        #--------------
        # LOGGING
        #--------------
        log_dict['validation_loss_per_batch'].append(val_loss.item())
      
     

      #--------------
      # VISUALISATION
      #--------------
      print(f'training_loss: {round(loss.item(), 4)} validation_loss: {round(val_loss.item(), 4)}\n')
      l = f'training_loss: {round(loss.item(), 4)} validation_loss: {round(val_loss.item(), 4)}\n'
      self.file1 = open("Train_logs_classification.txt",'a')
      self.file1.write(l)
      self.file1.close()
      # for test_images in test_loader:
      #   #  sending test images to device
      #   test_images = test_images.to(device)
      #   with torch.no_grad():
      #     #  reconstructing test images
      #     reconstructed_imgs = self.network(test_images)
      #   #  sending reconstructed and images to cpu to allow for visualization
      #   reconstructed_imgs = reconstructed_imgs.cpu()
      #   test_images = test_images.cpu()

      #   #  visualisation
      #   imgs = torch.stack([test_images.view(-1, 3, 768, 768), reconstructed_imgs], 
      #                     dim=1).flatten(0,1)
      #   grid = make_grid(imgs, nrow=10, normalize=True, padding=1)
      #   grid = grid.permute(1, 2, 0)
      #   plt.figure(dpi=170)
      #   plt.title('Original/Reconstructed')
      #   plt.imshow(grid)
      #   log_dict['visualizations'].append(grid)
      #   plt.axis('off')
      #   plt.show()
    
    # self.file1.close() 
    return log_dict

  def autoencode(self, x):
    return self.network(x)

  def encode(self, x):
    encoder = self.network.encoder
    return encoder(x)
  
  def decode(self, x):
    decoder = self.network.decoder
    return decoder(x)

In [25]:
classification_model.to(device)

Sequential(
  (0): Encoder(
    (net): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): ReLU()
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU()
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (9): ReLU()
      (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): Flatten(start_dim=1, end_dim=-1)
      (13): Linear(in_features=2359296, out_features=200, bias=True)
      (14): ReLU()
    )
  )
  (1): Sequential(
    (0): Linear(in_features=200, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=5, bias=True)
  )
)

In [26]:
with Image.open('/home/21je0963/Competiton/data/train_img/56947_thumbnail.png') as imgg:
    imgg = transform(imgg).to(device)
    output = classification_model(imgg)
output

tensor([[ 1.2500,  1.3625, -1.0828, -0.2255, -3.7909]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [27]:
output.shape

torch.Size([1, 5])

In [ ]:
modell = ConvolutionalAutoencoder(classification_model)
log_dict = modell.train(nn.MSELoss(), epochs=10, batch_size=4, training_set=train_custom_dataset, validation_set=val_custom_dataset)

Epoch 1/10
training...


100%|██████████| 54/54 [01:40<00:00,  1.87s/it]


validating...


100%|██████████| 14/14 [00:24<00:00,  1.75s/it]


training_loss: 0.1397 validation_loss: 0.1137

Epoch 2/10
training...


...

100%|██████████| 14/14 [00:24<00:00,  1.75s/it]


training_loss: 0.1576 validation_loss: 0.1438

Epoch 4/10
training...


100%|██████████| 54/54 [01:32<00:00,  1.71s/it]


validating...


 14%|█▍        | 2/14 [00:05<00:34,  2.84s/it]